# Линейная классификация

Будем решать задачу оттока клиентов телеком оператора.

In [8]:
from matplotlib.pylab import rc, plot
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

random_state=0
df = pd.read_csv('telecom_churn.csv')

d = {'Yes' : 1, 'No' : 0}

df['International plan'] = df['International plan'].map(d)
df['Voice mail plan'] = df['Voice mail plan'].map(d)
df['Churn'] = df['Churn'].astype('int64')
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [9]:
#df=df.drop('State',axis=1)
df.head()
y=df['Churn']
X=df.drop('Churn',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,
                                                    random_state=0)

### Масштабирование признаков

Будем масштабировать признаки перед обучением модели. Это, среди, прочего, сделает нашу регуляризацию более честной: теперь все признаки будут регуляризоваться в равной степени.  Кроме того, можно будет оценивать вклад каждого признака в модель по его весу в модели.

Для этого воспользуемся трансформером  `StandardScaler`. Трансформеры в sklearn имеют методы `fit` и `transform` (а еще `fit_transform`). Метод `fit` принимает на вход обучающую выборку и считает по ней необходимые значения (например статистики, как `StandardScaler`: среднее и стандартное отклонение каждого из признаков). transform применяет преобразование к переданной выборке.

Пропущенные значения закодируем отдельной категорией, пропущенные численные - средним значением признака по тестовой выборке.Применим `StandartScaler` к численным признакам и `OneHotEncoder`- к категориальным. Применим `ColumnTransformer` для единообразной трансформации обучающей и тестовой выборки.

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

numeric_data = X_train.select_dtypes([np.number])
numeric_data_mean = numeric_data.mean()
X_train = X_train.fillna(numeric_data_mean)
X_test = X_test.fillna(numeric_data_mean)

numeric_features = numeric_data.columns
categorical = list(X_train.dtypes[X_train.dtypes == "object"].index)
X_train[categorical] = X_train[categorical].fillna("NotGiven")
X_test[categorical] = X_test[categorical].fillna("NotGiven")

### Кодирование категориальных признаков

Нам нужно закодировать категориальные признаки числами, ведь линейная модель может работать только с численными значениями. Два стандартных трансформера из sklearn для работы с категориальными признаками — `OrdinalEncoder` (просто перенумеровывает значения признака натуральными числами) и `OneHotEncoder`(dummy-признаки).

`OneHotEncoder` ставит в соответствие каждому признаку целый вектор, состоящий из нулей и одной единицы (которая стоит на месте, соответствующем принимаемому значению, таким образом кодируя его).

Нужно ли применять скалирующий трансформатор к признакам, закодированным `OneHotEncoder`.

Пользоваться OrdinalEncoder в случае линейной модели —  плохой вариант. 

Недостатки OneHot-кодирования.

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error

column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('scaling', StandardScaler(), numeric_features)
])
X_train_encoded=column_transformer.fit_transform(X_train)
pd.DataFrame(X_train_encoded.toarray()).head()

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.879648,-0.653460,-0.879573,-0.377791,-1.214353,-0.379791,-0.334364,-0.605667,-0.336336,1.097125
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.781652,-0.403801,-0.782731,-0.174258,0.602831,-0.173408,-0.049965,-0.605667,-0.046662,1.097125
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.007043,0.295246,-1.006391,-0.976535,-0.204806,-0.976985,-0.227714,-1.425524,-0.231000,0.338190
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.519022,0.145451,-0.519873,-0.332342,0.602831,-0.331489,-1.720811,0.214190,-1.718868,-0.420745
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.468778,1.643408,0.469303,-0.294797,-1.214353,-0.296360,-1.471962,0.214190,-1.468696,0.338190


### Pipeline

Обучение модели часто представляется как последовательность некоторых действий с обучающей и тестовой выборками (например, сначала нужно отмасштабировать выборку (причем для обучающей выборки нужно применить метод fit, а для тестовой — transform), а затем обучить/применить модель (для обучающей fit, а для тестовой — predict). Класс `sklearn.pipeline.Pipeline` позволяет хранить эту последовательность шагов и корректно обрабатывает разные типы выборок: и обучающую, и тестовую.

В sklearn еще есть класс для ускоренного формирования последовательности без именования: `sklearn.pipeline.make_pipeline`

### Логистическая регрессия

class sklearn.linear_model.LogisticRegressionCV(*,

                     Cs=10, fit_intercept=True, cv=None, dual=False, penalty='l2',
                     
                     scoring=None,  solver='lbfgs', tol=0.0001, max_iter=100,
                     
                     class_weight=None, n_jobs=None, verbose=0, refit=True, 
                     
                     intercept_scaling=1.0, multi_class='auto', random_state=None, l1_ratios=None)
                     
   - Cs - параметр, обратный коэфф.регуляризации
   - penalty - тип регуляризации: ‘l1’, ‘l2’, ‘elasticnet’
   - cv : число блоков кроссвалидации или генератор кросс-валидации (например,KFold или LeaveOneOut) 
   
Кроме стандарных методов fit,predict  полезен метод predict_proba()

Создадим в Pipeline с трансформацией признаков логиcтическую регрессию с L2- регуляризацией, найдем наилучшие параметры на кросс-валидации по сетке параметра регуляризации С: [0.0001,0.001,0.01,0.1,1,10,100].
Используем класс LogisticRegressionCV, random_state=42 и число блоков кросс-валидации cv=5


In [13]:
import warnings
warnings.simplefilter("ignore")

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression', LogisticRegressionCV(penalty='l2',Cs=[0.0001,0.001,0.01,0.1,1,10,100],
                                        cv=5,refit=True))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)
#predict_proba возвращает вероятности классов 
y_proba = model.predict_proba(X_test)
print(pd.DataFrame(y_proba[:, 1]).head())
print(pd.DataFrame(y_proba).head())

          0
0  0.275122
1  0.014658
2  0.223864
3  0.054583
4  0.883578
          0         1
0  0.724878  0.275122
1  0.985342  0.014658
2  0.776136  0.223864
3  0.945417  0.054583
4  0.116422  0.883578


In [14]:

from sklearn.metrics import accuracy_score
print("1. Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("2. C = %.4f" % model[1].C_)


1. Test accuracy = 0.8396
2. C = 100.0000


Как можно улучшить качество линейной модели

### Биннаризация признаков

Для биннаризации признаков можно применять класс `sklearn.preprocessing.KBinsDiscretizer`:
sklearn.preprocessing.KBinsDiscretizer(n_bins=5, *, encode='onehot', strategy='quantile', dtype=None)

       strategy(default=’quantile’):
            - uniform -равномерное разбиение  значений признака
            - quantile - по кванитилям
            - kmeans - 1D k-means cluster.
            - encode: 'ordinal'
Преимущества биннаризации: улавливание немонотонных (vs Feature Engineering) и нелинейных зависимостей отклика от признака. 
Когда имеет смысл использовать ordinal-кодирование?
       

Вместо `StandardScaler` применим к численным признакам метод класса `sklearn.preprocessing.KBinsDiscretizer` с разбиением на 25 групп и стратегией разбиения 'kmeans' к численным признакам, а `OneHotEncoder`применим к категориальным признакам. 

Применим `ColumnTransformer` для единообразной трансформации обучающей и тестовой выборки.
Создадим в pipeline с трансформацией признаков логистическую регрессию с L2- регуляризацией, найдем наилучшие параметры на кросс-валидации по сетке параметра регуляризации С: [0.0001,0.001,0.01,0.1,1,10,100].


In [17]:
from sklearn.preprocessing import KBinsDiscretizer

In [22]:


column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('binner',  KBinsDiscretizer(n_bins=25, strategy='kmeans'), numeric_features)
])

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression',  LogisticRegressionCV(penalty='l2',Cs=[0.0001,0.001,0.01,0.1,1,10,100],cv=5,
                                       random_state=random_state))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("C= %.4f" % model[1].C_)

Test accuracy = 0.8816
C= 1.0000


попробуем квантильную стратегиею разбиения признака (получается одинаковое кол-во значений признака в каждой группе)

In [23]:
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('binner',  KBinsDiscretizer(n_bins=25, strategy='quantile'), numeric_features)
])

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression',  LogisticRegressionCV(penalty='l2',Cs=[0.0001,0.001,0.01,0.1,1,10,100],cv=5,
                                       random_state=random_state))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Test accuracy = %.4f" % accuracy_score(y_pred,y_test))
print("C= %.4f" % model[1].C_)


Test accuracy = 0.8771
C= 0.1000


### Стохастический градиентый спуск в задачах классификации


Решим задачу методом стохастического градиентого спуска,
применяя такую же трансформацю признаков, как в последней задаче:
    - KBinsDiscretizer с разбиением на 25 групп и стратегией разбиения 'kmeans' к численным признакам,
    - OneHotEncoder- к категориальным.
Воспользуемся классом моделей градиентного спуска
`sklearn.linear_model.SGDClassifier` с параметрами:
- learning_rate='constant'
- max_iter=20 (сколько раз каждый объект случайно выбирается для модификации весов)
- loss='log'
- alpha=0.0001 (сила регуляризации)
- penalty='l2'
-  сетка значений шага скорости обучения epsilon (learning rate): [0.001,0.01,0.05,0.1,0.2,0.5,1.0,1.5,5,10] (Возьмем достаточно большие значения для иллюстрации расходимости)

SGDRegressor(loss='squared_error', penalty='l2') решает ту же задачу, что и Ridge() (др.solver)

Просто и быстро работает.

Чувствителен к масштабу признаков, требует гиперпараметры, от которых может заметно зависеть качество (регуляризация, max_iter)


In [30]:
results=[]
for eps in [0.001,0.01,0.05,0.1,0.2,0.5,1.0,1.5,5,10]:
    from sklearn.linear_model import SGDClassifier
    pipeline = Pipeline(steps=[
        ('ohe_and_scaling', column_transformer),
        ('regression', SGDClassifier(max_iter=20,loss='log',penalty='l2',alpha=0.001, 
                                     learning_rate='constant',eta0=eps,
                                     random_state=random_state,n_iter_no_change=20))
    ])

    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(" Test accuracy = %.4f learning rate= %.4f" % (accuracy_score(y_pred,y_test), eps))
    results.append((accuracy_score(y_pred,y_test), eps))


 Test accuracy = 0.8696 learning rate= 0.0010
 Test accuracy = 0.8681 learning rate= 0.0100
 Test accuracy = 0.8741 learning rate= 0.0500
 Test accuracy = 0.8651 learning rate= 0.1000
 Test accuracy = 0.8006 learning rate= 0.2000
 Test accuracy = 0.6237 learning rate= 0.5000
 Test accuracy = 0.5457 learning rate= 1.0000
 Test accuracy = 0.5172 learning rate= 1.5000
 Test accuracy = 0.7436 learning rate= 5.0000
 Test accuracy = 0.8381 learning rate= 10.0000


Видим, как при  слишком большом learning rate SGD не сходится. Не достигает такого же качества как просто LogisticRegression()

In [31]:
print("Max test accuracy = %.4f \nlearning rate= %.4f" % 
      (max(results, key = lambda i : i[0])[0],max(results, key = lambda i : i[0])[1]))

Max test accuracy = 0.8741 
learning rate= 0.0500


Полностью аналогично предыдущей задаче обучим модель с параметром learning_rate='adaptive'(делит eps на 5, если нет улучшения  training loss на нескольких итерациях(число задается другими параметрами). Если задать слишком большой eps, то очень возможно не справится, зависит, в частности, от параметра n_iter_no_change и др.


In [32]:
results=[]
for eps in [1]:
    from sklearn.linear_model import SGDClassifier
    pipeline = Pipeline(steps=[
        ('ohe_and_scaling', column_transformer),
        ('regression', SGDClassifier(max_iter=20,loss='log',penalty='l2',alpha=0.001,
                                     learning_rate='adaptive',eta0=eps,
                                     random_state=random_state,n_iter_no_change=5 ))
    ])

    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(eps,accuracy_score(y_pred,y_test),model[1].n_iter_)
    results.append((accuracy_score(y_pred,y_test), eps))

1 0.8680659670164917 20


Адаптивный сам подстраивается, но немного дальше от оптимума

Что еще можно сделать с линейной классификацией: 
 - сегментация выборки и построение нескольких моделей на сегментах
 - Feature Engineering (PolynomialFeatures итп)
 

### Многоклассовая классификация

<center>
<img src="./images/one_vs.png" width=500/>
</center>

#### Один против всех(one versus rest),One versus one

One-vs-rest (OvR) Обучаем несколько бинарных классификаторов, предсказываем наиболее вероятный класс

В методе отмечаем параметр  `multi_class` (Например, в `LogisticRegression`)

In [99]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

model = LogisticRegression(multi_class='ovr')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)


In [100]:
X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1000, 10), (1000,), (670, 10), (670,), (330, 10), (330,))

In [101]:
pd.DataFrame(y_proba)

,0,1,2
0,0.468237,0.153721,0.378043
1,0.533255,0.173236,0.293509
2,0.260221,0.086506,0.653273
3,0.401586,0.250503,0.347911
4,0.645968,0.051456,0.302576
...,...,...,...
325,0.504307,0.098508,0.397185
326,0.462747,0.335996,0.201257
327,0.043586,0.636665,0.319749
328,0.357171,0.028820,0.614010
